In [ ]:
import numpy as np

def dB_to_lin(pow_dB):
    return 10**(np.array(pow_dB)/10)

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))

rssi_3470_3510 = np.array([-85.53,
-95.92,
-96.29,
-101.19,
-93.44,
-99.88,
-88.62,
-99.78,
-96.45,
-88.8
])


rssi_3610_3650 = np.array([-84.93,
-98.39,
-101.84,
-101.58,
-92.01,
-101.67,
-86.77,
-98.45,
-102.38,
-89.61
])

rssi_2160_2170 = np.array([-85.43,
-76.88,
-76.22,
-86.68,
-78.85,
-80.78,
-79.53,
-74.71,
-80.88,
-72.33
])


In [ ]:
import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[::10, ::10]

N1 = map_.shape[0]
N2 = map_.shape[1]
en = SLC[column_map["axis"]]

UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)

UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)

### Implement Pseudo-Inverse

In [ ]:
data_points = np.array([
    [966, 2992],
    [2569, 3767],
    [2873, 3447],
    [2621, 4286],
    [1312, 3830],
    [3828, 2667],
    [242, 2442],
    [1711, 3145],
    [2852, 1584],
    [1903, 2393]
])
data_points = data_points.astype(float)/10.0
data_points = np.floor(data_points).astype("int")

In [ ]:
def serialize_index(row, col, num_cols=map_.shape[1]):
    return row * num_cols + col

dp_serial = []
for dp in data_points:
    serial_dp = serialize_index(dp[1], dp[0])
    dp_serial.append(serial_dp)
    
dp_serial = np.array(dp_serial)
dp_serial

In [ ]:
from tqdm import tqdm
def deserialize_index(serialized_index, num_cols=map_.shape[1]):
    row = serialized_index // num_cols
    col = serialized_index % num_cols
    return row, col

def euclidean_distance(serialized_index1, serialized_index2, num_cols=map_.shape[1]):
    row1, col1 = deserialize_index(serialized_index1, num_cols)
    row2, col2 = deserialize_index(serialized_index2, num_cols)

    distance = ((row2 - row1)**2 + (col2 - col1)**2)**0.5*(0.5*10)
    if distance<1:
        distance=1
    return distance

In [ ]:
band = "3610_3650"
minimizer_Ts = np.load("minimizer_T_"+band+"_low_res3.npy")
probabilities = np.load("f_xz_2d_"+band+"_low_res_cov3.npy")
signal_estimates = np.load("signal_estimates_"+band+"_low_res_cov3.npy")
if band=="2160_2170":
    signal_strengths = rssi_2160_2170
elif band=="3470_3510":
    signal_strengths = rssi_3470_3510
elif band=="3610_3650":
    signal_strengths = rssi_3610_3650

### Probabilities

In [ ]:
import matplotlib
#signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)

matplotlib.rcParams['font.family'] = 'Times New Roman'

plt.figure(figsize=(13, 8))
plt.contourf(X, Y, minimizer_Ts, 100, cmap='hot')  # Use the masked array here
#plt.colorbar(label='Predicted Signal Strength [dBX]')
# Adjust the position of the colorbar on the right to be closer to the figure
cbar = plt.colorbar(label='Probability')
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='Estimated Transmit Power [dBX]', size=18)

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=150, edgecolor='green', linewidth=2, cmap='hot', label='Spectrum Monitoring Locations')
scatter_cbar = plt.colorbar(scatter, label='Observed Signal Strength [dBX]', location='left')  # Colorbar for signal strengths on the left
scatter_cbar.ax.tick_params(labelsize=18)
scatter_cbar.set_label(label='Observed Signal Strength [dBX]', size=18)
plt.legend()

interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Set the tick values and labels
#plt.yticks(ticks=tick_values, labels=tick_labels, fontsize=10, rotation=90)
# Adjust the position of the colorbars
cbar.ax.set_position([0.77, 0.1, 0.04, 0.8])  # Adjust the main colorbar closer to the plot
scatter_cbar.ax.set_position([0.18, 0.1, 0.04, 0.8])  # Adjust the scatter colorbar to avoid overlap

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



plt.xlabel('UTM$_E$ [m]',fontsize=18, labelpad=10)
plt.ylabel('UTM$_N$ [m]',fontsize=18, labelpad=10)
plt.title('Estimated Transmit Power @ '+band.split('_')[0]+'-'+ band.split('_')[1]+' MHz', fontsize=20)
plt.show()

In [ ]:
signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)

matplotlib.rcParams['font.family'] = 'Times New Roman'

plt.figure(figsize=(13, 8))
plt.contourf(X, Y, probabilities, 100, cmap='hot')  # Use the masked array here
#plt.colorbar(label='Predicted Signal Strength [dBX]')
# Adjust the position of the colorbar on the right to be closer to the figure
cbar = plt.colorbar(label='Predicted Signal Strength [dBX]')
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='Probability Mass', size=18)

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=150, edgecolor='green', linewidth=2, cmap='hot', label='Spectrum Monitoring Locations')
scatter_cbar = plt.colorbar(scatter, label='Observed Signal Strength [dBX]', location='left')  # Colorbar for signal strengths on the left
scatter_cbar.ax.tick_params(labelsize=18)
scatter_cbar.set_label(label='Observed Signal Strength [dBX]', size=18)
plt.legend()

interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Set the tick values and labels
#plt.yticks(ticks=tick_values, labels=tick_labels, fontsize=10, rotation=90)
# Adjust the position of the colorbars
cbar.ax.set_position([0.77, 0.1, 0.04, 0.8])  # Adjust the main colorbar closer to the plot
scatter_cbar.ax.set_position([0.18, 0.1, 0.04, 0.8])  # Adjust the scatter colorbar to avoid overlap

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



plt.xlabel('UTM$_E$ [m]',fontsize=18, labelpad=10)
plt.ylabel('UTM$_N$ [m]',fontsize=18, labelpad=10)
plt.title('2D PMF of Transmitter Location @ '+band.split('_')[0]+'-'+ band.split('_')[1]+' MHz', fontsize=20)
plt.show()

### Signal Strength Estimates

In [ ]:
signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)

min_strength = min(lin_to_dB(signal_estimates).min(), signal_strengths.min())
max_strength = max(lin_to_dB(signal_estimates).max(), signal_strengths.max())

colorbar_ticks = np.linspace(min_strength, max_strength, num=7)  # Adjust the number of ticks as needed
colorbar_tick_labels = ['{:.1f}'.format(tick) for tick in colorbar_ticks]

matplotlib.rcParams['font.family'] = 'Times New Roman'

plt.figure(figsize=(13, 8))
plt.contourf(X, Y, lin_to_dB(signal_estimates), 100, cmap='hot', vmin=min_strength, vmax=max_strength)  # Use the masked array here
#plt.colorbar(label='Predicted Signal Strength [dBX]')
# Adjust the position of the colorbar on the right to be closer to the figure
cbar = plt.colorbar(label='Predicted Signal Strength [dBX]')
cbar.ax.tick_params(labelsize=18)
cbar.set_ticks(colorbar_ticks)  # Set the ticks
cbar.set_label(label='Predicted Signal Strength [dBX]', size=18)
cbar.set_ticklabels(colorbar_tick_labels)  # Set the tick labels

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=150, edgecolor='green', linewidth=2, cmap='hot', label='Spectrum Monitoring Locations', vmin=min_strength, vmax=max_strength)
scatter_cbar = plt.colorbar(scatter, label='Observed Signal Strength [dBX]', location='left')  # Colorbar for signal strengths on the left
scatter_cbar.ax.tick_params(labelsize=18)
scatter_cbar.set_ticks(colorbar_ticks)  # Set the ticks
scatter_cbar.set_label(label='Observed Signal Strength [dBX]', size=18)
scatter_cbar.set_ticklabels(colorbar_tick_labels)  # Set the tick labels
plt.legend()

interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Set the tick values and labels
#plt.yticks(ticks=tick_values, labels=tick_labels, fontsize=10, rotation=90)
# Adjust the position of the colorbars
cbar.ax.set_position([0.77, 0.1, 0.04, 0.8])  # Adjust the main colorbar closer to the plot
scatter_cbar.ax.set_position([0.18, 0.1, 0.04, 0.8])  # Adjust the scatter colorbar to avoid overlap

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



plt.xlabel('UTM$_E$ [m]',fontsize=18, labelpad=10)
plt.ylabel('UTM$_N$ [m]',fontsize=18, labelpad=10)
plt.title('2D Predictions of Signal Strength @ '+band.split('_')[0]+'-'+ band.split('_')[1]+' MHz', fontsize=20)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
signal_estimates = np.reshape(signal_estimates, map_.shape)
scores = []
for i in range(len(data_points)):
    train_points = np.delete(data_points, i, axis=0)
    train_strengths = np.delete(signal_strengths, i, axis=0)
    test_point = data_points[i, :].reshape(1, -1)
    test_strength = signal_strengths[i]
    predicted_strength = lin_to_dB(signal_estimates[data_points[i][1], data_points[i][0]])
    mse = (test_strength-predicted_strength)**2
    scores.append(mse)
print(np.mean(scores))
print(np.std(scores))

In [ ]:
np.std(signal_strengths)**2